# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [ ]:
archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

palabra = input("Escribe la palabra que quieres buscar: ").lower()

# Revisar en qué archivos aparece la palabra
for archivo in archivos:
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            contenido = f.read().lower()
            if palabra in contenido:
                print(f"La palabra está en: {archivo}")
    except FileNotFoundError:
        print(f"No se encontró el archivo: {archivo}")

La palabra está en: ../data/01_corpus_turismo.txt
La palabra está en: ../data/01_corpus_turismo_500.txt


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [2]:
import os
import re
from collections import defaultdict

archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

# Crear el índice invertido
indice_invertido = defaultdict(set)

for archivo in archivos:
    if os.path.exists(archivo):
        with open(archivo, 'r', encoding='utf-8') as f:
            texto = f.read().lower()
            palabras = re.findall(r'\b\w+\b', texto)
            for palabra in palabras:
                indice_invertido[palabra].add(archivo)

palabra = input("Ingresa la palabra a buscar: ").lower()

# Buscar en el índice
if palabra in indice_invertido:
    print("La palabra aparece en los documentos:")
    for doc in indice_invertido[palabra]:
        print(doc)
else:
    print("La palabra no se encontró en ninguno de los documentos.")


La palabra aparece en los documentos:
../data/01_corpus_turismo.txt
../data/01_corpus_turismo_500.txt


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [ ]:
import os
import re
import time
from collections import defaultdict

archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

indice_invertido = defaultdict(set)

for archivo in archivos:
    if os.path.exists(archivo):
        with open(archivo, 'r', encoding='utf-8') as f:
            texto = f.read().lower()
            palabras = re.findall(r'\b\w+\b', texto)
            for palabra in palabras:
                indice_invertido[palabra].add(archivo)

palabras_a_buscar = ['quito', 'montañita', 'feriado', 'playas', 'aventura', 'galápagos']
resultados_tiempos = []

for palabra in palabras_a_buscar:
    palabra = palabra.lower()

    # Medir tiempo búsqueda lineal
    inicio1 = time.perf_counter()
    for archivo in archivos:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read().lower()
                if palabra in contenido:
                    break
    fin1 = time.perf_counter()
    tiempo_lineal = fin1 - inicio1

    # Medir tiempo búsqueda con índice
    inicio2 = time.perf_counter()
    _ = list(indice_invertido.get(palabra, []))
    fin2 = time.perf_counter()
    tiempo_indice = fin2 - inicio2

    resultados_tiempos.append((palabra, tiempo_lineal, tiempo_indice))

print("\nTabla Comparativa de Tiempos de Búsqueda:\n")
print("{:<12} {:>20} {:>20}".format("Palabra", "Lineal (s)", "Índice (s)"))
print("-" * 60)
for palabra, t_lineal, t_indice in resultados_tiempos:
    print("{:<12} {:>20.6f} {:>20.6f}".format(palabra, t_lineal, t_indice))



Tabla Comparativa de Tiempos de Búsqueda:

Palabra                Lineal (s)           Índice (s)
------------------------------------------------------------
quito                    0.000284             0.000004
montañita                0.000181             0.000002
feriado                  0.000169             0.000002
playas                   0.000161             0.000001
aventura                 0.000177             0.000001
galápagos                0.000174             0.000001


## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [4]:
import os
import re
import time
from collections import defaultdict

archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']
indice_invertido = defaultdict(set)

for archivo in archivos:
    if os.path.exists(archivo):
        with open(archivo, 'r', encoding='utf-8') as f:
            texto = f.read().lower()
            palabras = re.findall(r'\b\w+\b', texto)
            for palabra in palabras:
                indice_invertido[palabra].add(archivo)

# Leer términos de búsqueda
entrada = input("Ingresa las palabras a buscar (separadas por espacio): ")
terminos = entrada.strip().lower().split()

# Búsqueda múltiple lineal
inicio_lineal = time.perf_counter()
resultados_lineal = []
for archivo in archivos:
    if os.path.exists(archivo):
        with open(archivo, 'r', encoding='utf-8') as f:
            texto = f.read().lower()
            if all(t in texto for t in terminos):
                resultados_lineal.append(archivo)
fin_lineal = time.perf_counter()

# Búsqueda múltiple con índice
inicio_indice = time.perf_counter()
resultados_indice = None
for termino in terminos:
    archivos_que_tienen = indice_invertido.get(termino, set())
    if resultados_indice is None:
        resultados_indice = archivos_que_tienen.copy()
    else:
        resultados_indice = resultados_indice.intersection(archivos_que_tienen)
if resultados_indice is None:
    resultados_indice = set()
fin_indice = time.perf_counter()

# Cálculo de tiempos
tiempo_lineal = fin_lineal - inicio_lineal
tiempo_indice = fin_indice - inicio_indice
speedup = tiempo_lineal / tiempo_indice if tiempo_indice > 0 else float('inf')

print("\nResultados de búsqueda:\n")
print("Búsqueda Lineal:", resultados_lineal)
print("Búsqueda con Índice:", list(resultados_indice))

print("\nTiempos:")
print(f"Búsqueda Lineal: {tiempo_lineal:.6f} segundos")
print(f"Búsqueda con Índice: {tiempo_indice:.6f} segundos")
print(f"Speedup: {speedup:.2f} veces más rápido usando índice invertido")


Resultados de búsqueda:

Búsqueda Lineal: ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']
Búsqueda con Índice: []

Tiempos:
Búsqueda Lineal: 0.001444 segundos
Búsqueda con Índice: 0.000148 segundos
Speedup: 9.76 veces más rápido usando índice invertido
